In [60]:
push!(LOAD_PATH, "../src")
include("../src/lanczos.jl")

using .StochasticLanczos
using SparseArrays
using LinearAlgebra
using FastExpm  # For computing matrix exponentials as a benchmark
using Plots

Import the test case. This includes the sparse matrix A, along with the matrix function f(A) = exp(A), and its exact computation

In [61]:
include("test_case_1.jl");

Now test the implementation of the stochastic Lanczos quadrature by comparing the approximate calculation with the exact calculation of $\Omega^\top f(A)\Omega$ where $\Omega\in \mathbb{R}^{n\times b}$ is a matrix whose columns $\omega_i$ are random samples. Note that $b$ is the block size.

In [62]:
b = 2
Ω = randn(n,b)
# Ω = hcat([ ω/norm(ω) for ω in eachcol(Ω) ]...)

100×2 Matrix{Float64}:
  1.09326    2.0417
  0.290863   0.434524
 -0.244191   1.88453
 -1.81399    0.697823
  1.98753   -0.724414
 -0.807579   1.03341
  0.172832   0.589025
  0.103637   1.19889
  0.765282  -0.410368
 -1.05107    0.236017
  ⋮         
 -0.140933  -0.654601
  0.130983  -1.76893
  2.61635    0.0618015
 -0.611363   0.182416
 -1.44196   -0.927515
  0.627563  -1.04502
 -2.06517    0.0932792
  1.4812     1.20149
  0.111988  -0.11257

In [63]:
k = 10
T_k = lanczos(matvecA, Ω[:, 1], k)
eigen(T_k).vectors

10×10 Matrix{Float64}:
 -0.061229   -0.22927   -0.399036   …   0.288788     0.205724  0.0124897
  0.123083    0.351325   0.480344       0.458415     0.383893  0.0534855
 -0.2247     -0.412757  -0.314093       0.327248     0.421162  0.208167
  0.313231    0.313035  -0.0170269      0.0018563    0.263729  0.652945
 -0.410984   -0.234268   0.200964      -0.185636    -0.21547   0.675375
  0.543521    0.045759  -0.351451   …   0.00254912  -0.406633  0.25799
 -0.46717     0.268252   0.126602       0.270435    -0.392677  0.065898
  0.32391    -0.437179   0.231949       0.455015    -0.342722  0.016206
 -0.192156    0.413362  -0.414101       0.461259    -0.253346  0.00396855
  0.0899143  -0.254399   0.325398       0.268255    -0.12553   0.000861143

In [64]:
test = zeros(n,1)
test[:,1] = Ω[:,1]
T_k, _ = block_lanczos(matvecA, test, 10)
eigen(T_k).vectors

10×10 Matrix{Float64}:
  0.061229    0.22927   -0.399036   …   0.288788     0.205724  0.0124897
 -0.123083   -0.351325   0.480344       0.458415     0.383893  0.0534855
  0.2247      0.412757  -0.314093       0.327248     0.421162  0.208167
 -0.313231   -0.313035  -0.0170269      0.0018563    0.263729  0.652945
  0.410984    0.234268   0.200964      -0.185636    -0.21547   0.675375
 -0.543521   -0.045759  -0.351451   …   0.00254912  -0.406633  0.25799
  0.46717    -0.268252   0.126602       0.270435    -0.392677  0.065898
 -0.32391     0.437179   0.231949       0.455015    -0.342722  0.016206
  0.192156   -0.413362  -0.414101       0.461259    -0.253346  0.00396855
 -0.0899143   0.254399   0.325398       0.268255    -0.12553   0.000861143

In [65]:
k = 5
block_stochastic_lanczos_quadrature(f, matvecA, Ω, k, "single")

1.288254233898859e46

In [70]:
block_stochastic_lanczos_quadrature(f, matvecA, Ω, k, "block")

1.2883014031049468e46

In [67]:
exact = tr(Ω'*exactfA*Ω)

1.2883161118950007e46 + 0.0im